MarketCo

Problem Definition

Importing Libraries

In [1]:
#import libraries
from __future__ import division
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans

#do not show warnings
import warnings
warnings.filterwarnings("ignore")


#import machine learning related libraries
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split


Loading the data

In [2]:
data = pd.read_csv('../Data/data.csv')
target = pd.read_csv('../Data/train_labels.csv')
test = pd.read_csv('../Data/test_labels.csv')

,date,purchase_id,client_id,category_code,item_code,qty_sold,price,cost
0,2020-06-01,0,0,0,0,1.0,29.187140,32.780351
1,2020-06-01,1,1,1,1,1.0,120.499503,89.553040
2,2020-06-01,1,1,2,2,1.0,123.064992,116.300730
3,2020-06-01,1,1,3,3,1.0,37.783663,18.476901
4,2020-06-01,1,1,4,4,1.0,40.274335,38.107742


Data Understanding

In [3]:
print("The shape of the data is:" + str(data.shape))

The shape of the data is:(10664209, 8)


In [4]:
#Checking if there are null values and each feature's type.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10664209 entries, 0 to 10664208
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   date           object 
 1   purchase_id    int64  
 2   client_id      int64  
 3   category_code  int64  
 4   item_code      int64  
 5   qty_sold       float64
 6   price          float64
 7   cost           float64
dtypes: float64(3), int64(4), object(1)
memory usage: 650.9+ MB


In [5]:
#Checking of there are any null values at any feature column.
data.isnull().sum()

date             0
purchase_id      0
client_id        0
category_code    0
item_code        0
qty_sold         0
price            0
cost             0
dtype: int64

In [8]:
data['date'] = pd.to_datetime(data['date'])

In [9]:
data = data.drop_duplicates(keep='first', inplace=False, ignore_index=False).copy()

In [13]:
df_4m = data[(data.date <= datetime(2020,9,30)) & 
(data.date >= datetime(2020,6,1))].reset_index(drop=True)

df_next = data[(data.date > datetime(2020,9,30)) & 
               (data.date <= datetime(2020,10,24))].reset_index(drop=True)

In [14]:
df_next.head()

,date,purchase_id,client_id,category_code,item_code,qty_sold,price,cost
0,2020-10-01,2562,2483,52,6428,1.00,76.920344,52.591244
1,2020-10-01,2563,2484,14,104,1.00,179.800122,124.988006
2,2020-10-01,2564,2485,58,796,1.63,62.166841,67.764180
3,2020-10-01,2564,2485,52,1263,1.00,115.504785,68.637428
4,2020-10-01,2564,2485,52,6429,1.00,120.631012,71.683630


In [15]:
df_user = pd.DataFrame(df_4m['client_id'].unique())
df_user.columns = ['client_id']

In [17]:
#create a dataframe with client id and first purchase date in df_next
df_first_purchase = df_next.groupby('client_id').date.min().reset_index()
df_first_purchase.columns = ['client_id','MinPurchaseDate']

#create a dataframe with client id and last purchase date in df_4m
df_last_purchase = df_4m.groupby('client_id').date.max().reset_index()
df_last_purchase.columns = ['client_id','MaxPurchaseDate']

In [18]:
#merge two dataframes
df_purchase_dates = pd.merge(df_last_purchase,df_first_purchase,on='client_id',how='left')

In [20]:
#calculate the time difference in days:
df_purchase_dates['Diff_in_days'] = (df_purchase_dates['MinPurchaseDate'] - df_purchase_dates['MaxPurchaseDate']).dt.days

In [21]:
#merge with tx_user 
df_user = pd.merge(df_user, df_purchase_dates[['client_id','Diff_in_days']],on='client_id',how='left')

In [23]:
df_user.isnull().sum()

client_id           0
Diff_in_days    10697
dtype: int64